# Neural Network - Digital Recognition

The subject today is digit recognition using neural networks. More precisely, the task is to train a neural network so that it finds the digit in a given image of handwritten digit.
The first step is to split the MNIST dataset into a training set, a validation set, and a test set. During the training phase, the neural network is trained on the training set. Still during the training phase, the recognition score of the neural network is checked on a subset of the training set and on the validation set in order to detect potential overfitting. During the test set, the true classification score of the fully-connected neural network is computed on the test set.

In [1]:
# Needed packages
import os
import numpy as np
import random as rd
import tensorflow as tf
import tools.tools as tls
from tqdm import tqdm
import matplotlib.pyplot as plt

## I - Process the data and visualisation

### Import data

In [2]:
path = os.getcwd() + '/mnist/data/'

train_img = np.load(path+'training_images.npy')
train_labels = np.load(path+'training_labels.npy')

valid_img = np.load(path+'validation_images.npy')
valid_labels = np.load(path+'validation_labels.npy')

test_img = np.load(path+'test_images.npy')
test_labels = np.load(path+'test_labels.npy')

### Process the data

In [3]:
# img
train_img_new = train_img.reshape(train_img.shape[0],
                                  train_img.shape[1]**2).astype('uint64')
valid_img_new = valid_img.reshape(valid_img.shape[0],
                                  valid_img.shape[1]**2).astype('uint64')
test_img_new = test_img.reshape(test_img.shape[0],
                                test_img.shape[1]**2).astype('uint64')

# label
train_lab_new = np.array([np.argmax(train_labels[y])
                          for y in range(len(train_labels))])
valid_lab_new = np.array([np.argmax(valid_labels[y])
                          for y in range(len(valid_labels))])
test_lab_new = np.array([np.argmax(valid_labels[y])
                         for y in range(len(valid_labels))])

### Random permutation of the train set (for the training phase of the neural network)

In [4]:
permut_index = np.random.permutation(train_img_new.shape[0])

train_img = train_img[permut_index, :, :]
train_img_new = train_img_new[permut_index, :]
train_labels = train_labels[permut_index, :]
train_lab_new = train_lab_new[permut_index]

### Visualisation (exported in the folder 'img_observation')

In [5]:
# Visualisation
def visualize(path, name, X, y, nb_img=36):
    """Allows to vizualise 36 img and the labels"""
    vector_viz = [rd.randint(0, len(X)-1) for y in np.arange(nb_img)]
    # Img
    nb_vertically = int(np.sqrt(nb_img))
    name_img_out = path+'visualize_img' + '_' + str(name) + '.png'
    tls.visualize_grayscale_images(X[vector_viz, :, :, :],
                                   nb_vertically, name_img_out)

    # Labels
    name_lab_out = path+'visualize_labels' + '_' + str(name)
    labels = train_lab_new[vector_viz]
    labels = labels.reshape((nb_vertically, nb_vertically))

    file = open(name_lab_out, "w")
    labels_str = ''
    for line in labels:
        for elt in line:
            labels_str = labels_str + str(elt) + ' '
        labels_str = labels_str + '\n'
    file.write(labels_str)
    file.close()
    print('Visualisation exported')


path = os.getcwd() + '/img_observation/'
X = train_img
y = train_lab_new
visualize(path, 'obs', X, y, nb_img=36)

Visualisation exported


## II - Basic Neural Network

import the model within the file 'NeuralNetwork.py'

### Training the neural network

In [6]:
# Import the model
from NeuralNetwork import neural_net_model

In [7]:
# Input data
nb_images = train_img_new.shape[0]
nb_input = train_img_new.shape[1]
batch_size = 1000
nb_hidden = 16
nb_classes = 10
learning_rate = 0.5
nb_epoch = 10

The following function is used in order to define batches during the training (it is the equivalent of the next batch function of tf)

In [8]:
def get_batch_idx(i, batch_size, nb_images):
    # Input
    max_batch_size = int(nb_images/batch_size)
    j = i % max_batch_size
    return np.arange(j*batch_size, (j+1)*batch_size)

In [9]:
# Initialisation
X = tf.placeholder(tf.float32, [None, nb_input])
Y = tf.placeholder(tf.float32, [None, nb_classes])

p = neural_net_model(X, nb_input, batch_size, nb_hidden, nb_classes,
                     learning_rate)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(p),
                                              reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(p, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

acc_train = []
loss_train = []

In [10]:
# Training
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    try:
        saver.restore(sess, 'mnist_predictions.ckpt')
    except Exception:
        pass

    # train the model mini batch with 100 elements, for 1K times
    for epoch in tqdm(range(nb_epoch)):

        for i in range(batch_size):
            batch_index = get_batch_idx(i, batch_size, nb_images)
            batch_x, batch_y = (train_img_new[batch_index, :],
                                train_labels[batch_index])
            sess.run([train_step], feed_dict={X: batch_x, Y: batch_y})

        # Accuracy and cross entropy - Training phase
        loss, acc = sess.run([cross_entropy, accuracy],
                             feed_dict={X: valid_img_new, Y: valid_labels})
        loss_train.append(loss)
        acc_train.append(acc)
        print("\nEpoch", str(epoch), "Loss :", str(loss),
              "Accuracy :", str(acc))

    # Test
    # evaluate the accuracy of the model
    test_accuracy = sess.run(accuracy, feed_dict={X: test_img_new,
                                                  Y: test_labels})
    print("\nAccuracy on test set:", test_accuracy)

    # pred
    pred = sess.run(p, feed_dict={X: test_img_new, Y: test_labels})
    pred = np.array([np.argmax(pred[y]) for y in range(len(pred))])
    labels = np.array([np.argmax(test_labels[y])
                       for y in range(len(test_labels))])

    print("\nErrors (%):", np.sum(labels != pred)/len(pred), '%')

    # Save the model
    '''if input('Save model ? [Y/N]') == 'Y':
        import os
        saver.save(sess, os.getcwd() +
                   '/nn_saved_sessions/mnist_predictions.ckpt')
        print('Model Saved')'''

    # Close the session
    sess.close()

 10%|█         | 1/10 [00:04<00:41,  4.59s/it]


Epoch 0 Loss : 0.7647578 Accuracy : 0.766


 20%|██        | 2/10 [00:09<00:37,  4.64s/it]


Epoch 1 Loss : 0.6079846 Accuracy : 0.8


 30%|███       | 3/10 [00:13<00:32,  4.61s/it]

...


Epoch 7 Loss : 0.43249854 Accuracy : 0.873


 90%|█████████ | 9/10 [00:40<00:04,  4.44s/it]


Epoch 8 Loss : 0.40850416 Accuracy : 0.883


100%|██████████| 10/10 [00:44<00:00,  4.40s/it]


Epoch 9 Loss : 0.42103118 Accuracy : 0.885

Accuracy on test set: 0.8899

Errors (%): 0.1101 %


### Visualisation

#### Accuracy

In [11]:
# Visualisation of the accuracy
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), acc_train, label='accuracy', color='blue')
plt.title('Training phase')
plt.ylabel('accuracy - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Cross entrepoy

In [12]:
# Visualisation of the loss
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), loss_train, label='loss', color='red')
plt.title('Training phase')
plt.ylabel('loss - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

### Observations

## III - Evolved Neural Network

In [13]:
from NeuralNetwork import neural_net_model_evol

In [14]:
# Input data
nb_images = train_img_new.shape[0]
nb_input = train_img_new.shape[1]
batch_size = 1000
nb_hidden1 = 64
nb_hidden2 = 64
nb_classes = 10
nb_epoch = 10

# Drop out level
prob_1 = 0.7
prob_2 = 0.7

The following function is used in order to define batches during the training (it is the equivalent of the next batch function of tf)

In [15]:
def get_batch_idx(i, batch_size, nb_images):
    # Input
    max_batch_size = int(nb_images/batch_size)
    j = i % max_batch_size
    return np.arange(j*batch_size, (j+1)*batch_size)

In [16]:
# Initialisation
X = tf.placeholder(tf.float32, [None, nb_input])
Y = tf.placeholder(tf.float32, [None, nb_classes])
keep_prob_1 = tf.placeholder(tf.float32)
keep_prob_2 = tf.placeholder(tf.float32)

p = neural_net_model_evol(X, nb_input, nb_hidden1, nb_hidden2, nb_classes,
                          keep_prob_1, keep_prob_2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(p),
                                              reduction_indices=[1]))

correct_pred = tf.equal(tf.argmax(p, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

acc_train = []
loss_train = []
learn_rate = []

We then define a piecewise constant for learning rate in order not to have a constant value through the training phase

In [17]:
# Piecewise constant
min_lr = 0.1
max_lr = 0.5
nb_values = 10

global_step = tf.Variable(0, trainable=False)
boundaries = list(np.linspace(batch_size,
                              batch_size*nb_epoch, nb_values,
                              dtype=np.int32)[:-1])
values = list(np.round(np.linspace(max_lr, min_lr, nb_values), 2))
learning_rate_pc = tf.train.piecewise_constant(global_step, boundaries, values)

j = 0
# Passing global_step to minimize() will increment it at each step.
learning_step_pc = tf.train.GradientDescentOptimizer(learning_rate_pc).minimize(cross_entropy,
                                                                 global_step=global_step)

In [18]:
# Training
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    try:
        saver.restore(sess, 'mnist_predictions.ckpt')
    except Exception:
        pass

    # train the model mini batch with 100 elements, for 1K times
    for epoch in tqdm(range(nb_epoch)):

        for i in range(batch_size):
            # Update learning rate
            l_rate = sess.run(learning_rate_pc, {global_step: j})
            j += 1

            batch_index = get_batch_idx(i, batch_size, nb_images)
            batch_x, batch_y = (train_img_new[batch_index, :],
                                train_labels[batch_index])
            sess.run([learning_step_pc], feed_dict={X: batch_x, Y: batch_y,
                                                    keep_prob_1: prob_1,
                                                    keep_prob_2: prob_2})

        # Accuracy and cross entropy - Training phase
        loss, acc = sess.run([cross_entropy, accuracy],
                             feed_dict={X: valid_img_new, Y: valid_labels,
                                        keep_prob_1: 1.0,
                                        keep_prob_2: 1.0})
        loss_train.append(loss)
        acc_train.append(acc)
        learn_rate.append(l_rate)
        print("\nEpoch", str(epoch), "Loss :", str(loss),
              "Accuracy :", str(acc), "Learning rate:", l_rate)

    # Test
    # evaluate the accuracy of the model
    test_accuracy = sess.run(accuracy, feed_dict={X: test_img_new,
                                                  Y: test_labels,
                                                  keep_prob_1: 1.0,
                                                  keep_prob_2: 1.0})
    print("\nAccuracy on test set:", test_accuracy)

    # pred
    pred = sess.run(p, feed_dict={X: test_img_new, Y: test_labels,
                                  keep_prob_1: 1.0, keep_prob_2: 1.0})
    pred = np.array([np.argmax(pred[y]) for y in range(len(pred))])
    labels = np.array([np.argmax(test_labels[y])
                       for y in range(len(test_labels))])

    print("\nErrors (%):", np.sum(labels != pred)/len(pred)*100, '%')

    # Save the model
    '''if input('Save model ? [Y/N]') == 'Y':
        import os
        saver.save(sess, os.getcwd() +
                   '/nn_saved_sessions/mnist_predictions.ckpt')
        print('Model Saved')'''

    # Close the session
    sess.close()

 10%|█         | 1/10 [00:06<01:00,  6.70s/it]


Epoch 0 Loss : 0.84792614 Accuracy : 0.747 Learning rate: 0.5


 20%|██        | 2/10 [00:13<00:53,  6.70s/it]


Epoch 1 Loss : 0.64333266 Accuracy : 0.802 Learning rate: 0.46


 30%|███       | 3/10 [00:20<00:46,  6.69s/it]

...


Epoch 7 Loss : 0.38811955 Accuracy : 0.886 Learning rate: 0.19


 90%|█████████ | 9/10 [01:00<00:06,  6.80s/it]


Epoch 8 Loss : 0.378854 Accuracy : 0.895 Learning rate: 0.14


100%|██████████| 10/10 [01:07<00:00,  6.79s/it]


Epoch 9 Loss : 0.37489542 Accuracy : 0.893 Learning rate: 0.1

Accuracy on test set: 0.8912

Errors (%): 10.879999999999999 %


### Visualisation

#### Accuracy

In [19]:
# Visualisation of the accuracy
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), acc_train, label='accuracy', color='blue')
plt.title('Training phase')
plt.ylabel('accuracy - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Loss

In [20]:
# Visualisation of the loss
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), loss_train, label='loss', color='red')
plt.title('Training phase')
plt.ylabel('loss - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Learning rate

In [21]:
# Visualisation of the learning rate
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), learn_rate, label='learning_rate', color='green')
plt.title('Training phase')
plt.ylabel('learning rate - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

### Observations

### We will now use a exponential decay for the learning rate

In [22]:
acc_train = []
loss_train = []
learn_rate = []

In [23]:
# Exponential decay of the learning rate
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.7
learning_rate_ed = tf.train.exponential_decay(starter_learning_rate,
                                              global_step, batch_size, 0.69,
                                              staircase=True)
j = 0
# Passing global_step to minimize() will increment it at each step.
learning_step_ed = tf.train.GradientDescentOptimizer(learning_rate_ed).minimize(cross_entropy,
                                                   global_step=global_step)

In [24]:
# initialize variables and session
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    try:
        saver.restore(sess, 'mnist_predictions.ckpt')
    except Exception:
        pass

    # train the model mini batch with 100 elements, for 1K times
    for epoch in tqdm(range(nb_epoch)):

        for i in range(batch_size):
            # Update learning rate
            l_rate = sess.run(learning_rate_ed, {global_step: j})
            j += 1

            batch_index = get_batch_idx(i, batch_size, nb_images)
            batch_x, batch_y = (train_img_new[batch_index, :],
                                train_labels[batch_index])
            sess.run([learning_step_ed], feed_dict={X: batch_x, Y: batch_y,
                                                    keep_prob_1: prob_1,
                                                    keep_prob_2: prob_2})

        # Accuracy and cross entropy - Training phase
        loss, acc = sess.run([cross_entropy, accuracy],
                             feed_dict={X: valid_img_new, Y: valid_labels,
                                        keep_prob_1: 1.0,
                                        keep_prob_2: 1.0})
        loss_train.append(loss)
        acc_train.append(acc)
        learn_rate.append(l_rate)
        print("\nEpoch", str(epoch), "Loss :", str(loss),
              "Accuracy :", str(acc), "Learning rate:", l_rate)

    # Test
    # evaluate the accuracy of the model
    test_accuracy = sess.run(accuracy, feed_dict={X: test_img_new,
                                                  Y: test_labels,
                                                  keep_prob_1: 1.0,
                                                  keep_prob_2: 1.0})
    print("\nAccuracy on test set:", test_accuracy)

    # pred
    pred = sess.run(p, feed_dict={X: test_img_new, Y: test_labels,
                                  keep_prob_1: 1.0, keep_prob_2: 1.0})
    pred = np.array([np.argmax(pred[y]) for y in range(len(pred))])
    labels = np.array([np.argmax(test_labels[y])
                       for y in range(len(test_labels))])

    print("\nErrors (%):", np.sum(labels != pred)/len(pred)*100, '%')

    # Save the model
    '''if input('Save model ? [Y/N]') == 'Y':
        import os
        saver.save(sess, os.getcwd() +
                   '/nn_saved_sessions/mnist_predictions.ckpt')
        print('Model Saved')'''

    # Close the session
    sess.close()

 10%|█         | 1/10 [00:06<01:00,  6.70s/it]


Epoch 0 Loss : 0.7083188 Accuracy : 0.786 Learning rate: 0.7


 20%|██        | 2/10 [00:13<00:53,  6.66s/it]


Epoch 1 Loss : 0.5790196 Accuracy : 0.816 Learning rate: 0.48299998


 30%|███       | 3/10 [00:19<00:46,  6.64s/it]

...


Epoch 7 Loss : 0.45251366 Accuracy : 0.87 Learning rate: 0.05212447


 90%|█████████ | 9/10 [00:59<00:06,  6.61s/it]


Epoch 8 Loss : 0.44641545 Accuracy : 0.873 Learning rate: 0.035965886


100%|██████████| 10/10 [01:06<00:00,  6.61s/it]


Epoch 9 Loss : 0.4415327 Accuracy : 0.865 Learning rate: 0.02481646

Accuracy on test set: 0.8843

Errors (%): 11.57 %


### Visualisation

#### Accuracy

In [25]:
# Visualisation of the accuracy
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), acc_train, label='accuracy', color='blue')
plt.title('Training phase')
plt.ylabel('accuracy - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Loss

In [26]:
# Visualisation of the loss
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), loss_train, label='loss', color='red')
plt.title('Training phase')
plt.ylabel('loss - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Learning rate

In [27]:
# Visualisation of the learning rate
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), learn_rate, label='learning_rate', color='green')
plt.title('Training phase')
plt.ylabel('learning rate - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

### Observations